This notebook was prepared to run on Google Colab, with the NYT API key placed in your gdrive as a csv file.

# DATA AND METHODS


In [ ]:
pip install pynytimes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import datetime
from dateutil.rrule import rrule, MONTHLY
import time
import pickle
import json
from datetime import date
from pynytimes import NYTAPI
import pandas as pd

# *EXTRACT - LOAD - TRANSFORM

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gdrive = '/content/drive/MyDrive/sample_data/'

In [ ]:
my_path = "MyDrive/sample_data/"

In [ ]:
my_key = open(gdrive + "NYTAPIkey.csv").read().strip()


 We extracted data with the query terms using the article_search() function. Although we can search through an article, NYT API only allows us to capture abstract, headline, lead_paragraph, and snippet.

In [ ]:
import itertools

nyt = NYTAPI(my_key, parse_dates=True)

#Person_list = ['Eric Garner', 'Michael Brown', 'Ahmaud Arbery', 'Rittenhouse']
Person_list = ['Black Lives Matter', 'BLM', 'BlackLivesMatter']
#type_material = ['News', 'Editorial', 'Op-Ed']
ll = []

#list_2021=['michael wayne jackson','arnell stewart','fanta bility','alvin motley','ryan leroux','winston smith','latoya denise james','andrew brown','makhia bryant','matthew williams','daunte wright','james lionel johnson','dominique williams','donovon lynch','marvin scott','jenoah donald','patrick warren','xzavier hill','robert howard','vincent belmonte']
#list_2020=['monica goods','bennie edwards','casey goodson','aiden ellison','quawan charles','kevin peterson','walter wallace','jonathan price','kurt reinhold','dijon kizzee','damian daniels','anthony mcclain','julian lewis','maurice abisid','brayla stone','rayshard brooks','priscilla slater','robert forbes','kamal flowers','jamel floyd','david mcatee','james scurlock','calvin horton','tony mcdade','dion johnson','george floyd','maurice gordon','cornelius fredericks','steven taylor','daniel prude','breonna taylor','barry gedeus','manuel ellis','reginald payne','ahmaud arbery','lionel morris','jaquyn light','william green','darius tarver','miciah lee']
#list_2020=['tony mcdade','dion johnson','george floyd','maurice gordon','cornelius fredericks','steven taylor','daniel prude','breonna taylor','barry gedeus','manuel ellis','reginald payne','ahmaud arbery','lionel morris','jaquyn light','william green','darius tarver','miciah lee']
#Person_list += list_2021
#Person_list += list_2020
#Person_list = list_2020
MAX_DATE = datetime.date(2021, 12, 31)
START_DATE = datetime.date(2019, 9, 1)
filters = {"police_violence": "body :((\"police brutality\") OR (\"police violence\") OR (\"police shooting\") OR (\"bullet\") OR (\"tear gas\") OR (\"unarmed black\") OR (\"chokehold\") OR (\"killing\") OR  (\"tase\") OR (\"murder\"))",
           "destruction":"body: ((\"burn\") OR (\"destroy\") OR (\"destruction\") OR (\"clash\") OR (\"chaos\") OR (\"loot\"))",
           "race": "body :((\"white\") OR (\"african american\") OR (\"race\") OR (\"brown\"))",
           "violent_protest":"body: ((\"riots\") OR (\"violent demonstration\") OR (\"violent protest\"))",
           "peaceful_protest":"body: ((\"peaceful protest\") OR (\"peaceful demonstration\") OR (\"nonviolent protest\") OR (\"non-violent protest\"))",
           "noquery": ""
}

#peaceful protest keywords: peaceful protest, nonviolent protest, non-violent protest, peaceful demonstration
#violent protest keywords: violent protest, riots, violent demonstration
#destruction vs police violence keywords:
#destruction keywords: burn, loot, destroy, destruction, clash, chaos
#police violence keywords: police violence, police brutality, police shooting, bullet, tear gas, tase, chokehold, unarmed black, murder, killing

for i in Person_list:
    timediff = datetime.timedelta(days=60)
    start_date = START_DATE
    end_date = min(MAX_DATE, start_date + timediff)
    all_articles = []
    while start_date < MAX_DATE:
        for key, fq in filters.items():
            if fq:
                options = {"sort": "oldest",
                            "fq": fq}
            else:
                options = {"sort": "oldest",}
            articles = nyt.article_search(
                query = '"'+ i + '"',
                results = 2000,
                dates = {
                  "begin": start_date,
                  "end": end_date
                },
                options = options
            )
            print(i, len(articles), start_date, end_date, key)
            all_articles.append((key, articles))
        start_date = end_date + datetime.timedelta(days=1)
        end_date = min(MAX_DATE, start_date + timediff)
    ll.append((i, all_articles))

/usr/local/lib/python3.8/dist-packages/pynytimes/helpers/article_search.py:16: UserWarning: Asking for a lot of results, because of rate limits it can take a while.
  warnings.warn(


Black Lives Matter 17 2019-09-01 2019-10-31 police_violence
Black Lives Matter 9 2019-09-01 2019-10-31 destruction
Black Lives Matter 27 2019-09-01 2019-10-31 race
Black Lives Matter 1 2019-09-01 2019-10-31 violent_protest
Black Lives Matter 2 2019-09-01 2019-10-31 peaceful_protest
Black Lives Matter 37 2019-09-01 2019-10-31 noquery
Black Lives Matter 20 2019-11-01 2019-12-31 police_violence
Black Lives Matter 17 2019-11-01 2019-12-31 destruction
Black Lives Matter 44 2019-11-01 2019-12-31 race
Black Lives Matter 6 2019-11-01 2019-12-31 violent_protest
Black Lives Matter 0 2019-11-01 2019-12-31 peaceful_protest
Black Lives Matter 51 2019-11-01 2019-12-31 noquery
Black Lives Matter 10 2020-01-01 2020-03-01 police_violence
Black Lives Matter 5 2020-01-01 2020-03-01 destruction
Black Lives Matter 20 2020-01-01 2020-03-01 race
Black Lives Matter 3 2020-01-01 2020-03-01 violent_protest
Black Lives Matter 1 2020-01-01 2020-03-01 peaceful_protest
Black Lives Matter 24 2020-01-01 2020-03-01 no


Here we extracted abstract, headline, lead_paragraph, and snippet from NYT articles.


In [ ]:
def articles_to_df(all_articles):
    l = []
    for person, tag_level in all_articles:
        for tag, articles in tag_level:
            for article in articles:
                text_list = []
                new_article = {
                    'filter': tag,
                    'abstract': article["abstract"],
                    "headline": article["headline"]["main"],
                    "lead_paragraph": article["lead_paragraph"],
                    "snippet": article["snippet"],
                    "pub_date": article["pub_date"],
                    "type_of_material": article.get("type_of_material"),
                    "person": person,
                    "web_url": article["web_url"],
                    '_id': article['_id']
                }
                for k in ["abstract", "headline", "lead_paragraph", "snippet"]:
                    v = new_article[k]
                    if v:
                        v = v.strip()
                    new_article[k] = v
                    if v not in text_list:
                        text_list.append(v)
                new_article["text"] = ' '.join(text_list)
                if any(
                      map(lambda person_name: person_name.lower() in new_article["text"].lower(),
                          Person_list)) or True:
                    l.append(new_article)
    data_df = pd.DataFrame(l)
    return data_df

data_df = articles_to_df(ll)
with open(gdrive+"nyt_articles1213.p", "wb") as f:
    f.write(pickle.dumps(data_df))
data_df

,filter,abstract,headline,lead_paragraph,snippet,pub_date,type_of_material,person,web_url,_id,text
0,police_violence,"The founder of a Chicago youth group, who shar...",A Gift From the N.F.L. Ignites a Firestorm Ove...,"For several years, a small nonprofit operating...","The founder of a Chicago youth group, who shar...",2019-09-06 22:44:19+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/09/06/arts/music/...,nyt://article/65b5b170-9325-53be-a95c-1edca8d2...,"The founder of a Chicago youth group, who shar..."
1,police_violence,"In an interview, Ms. Harris sought to paint he...",‘Trust Me’: Kamala Harris Makes Big Play on Cr...,Senator Kamala Harris of California released a...,"In an interview, Ms. Harris sought to paint he...",2019-09-09 09:30:07+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/09/09/us/politics...,nyt://article/009cafc5-8b75-5f39-a757-6e95a249...,"In an interview, Ms. Harris sought to paint he..."
2,police_violence,"Amber Guyger killed an unarmed black man, Both...",Former Dallas Police Officer Is Guilty of Murd...,DALLAS — The case was unusual from the very be...,"Amber Guyger killed an unarmed black man, Both...",2019-10-01 15:47:07+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/01/us/amber-gu...,nyt://article/2156cec9-000d-5c61-bf5f-bae2ce2b...,"Amber Guyger killed an unarmed black man, Both..."
3,police_violence,"In this week’s newsletter, Marc Stein has scoo...",The League They Told You Not to Worry About,Want more basketball in your inbox? Sign up fo...,"In this week’s newsletter, Marc Stein has scoo...",2019-10-02 16:39:23+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/02/sports/bask...,nyt://article/b1fca0ad-762b-52a9-8006-74599284...,"In this week’s newsletter, Marc Stein has scoo..."
4,police_violence,George Gascón resigned as San Francisco’s dist...,Two Prosecutors Were Shaped by 1980s Los Angel...,LOS ANGELES — Walking the beat as a cop in the...,George Gascón resigned as San Francisco’s dist...,2019-10-03 23:33:17+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/03/us/los-ange...,nyt://article/1b74b499-026e-5c3e-ac63-d9fe0ca9...,George Gascón resigned as San Francisco’s dist...
...,...,...,...,...,...,...,...,...,...,...,...
7987,noquery,Join us as we share stories of the Black exper...,From Many Comes One,,Join us as we share stories of the Black exper...,2021-07-22 18:26:42+00:00,None,BlackLivesMatter,https://www.nytimes.com/paidpost/starz/soul-of...,nyt://paidpost/ee158427-9a9b-5119-b874-72bd15d...,Join us as we share stories of the Black exper...
7988,destruction,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."
7989,race,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."
7990,noquery,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."


In [ ]:
#data_df.to_csv(gdrive+"victim_articlesv2.csv", encoding='utf-8')
data_df.to_csv(gdrive+"nyt_articles1213.csv", encoding='utf-8')

In [ ]:
#data_df = pd.read_csv(gdrive + "nyt_articles0913.csv")
data_df = pickle.load(open(gdrive+"nyt_articles1213.p", "rb"))

In [ ]:
data_df

,filter,abstract,headline,lead_paragraph,snippet,pub_date,type_of_material,person,web_url,_id,text
0,police_violence,"The founder of a Chicago youth group, who shar...",A Gift From the N.F.L. Ignites a Firestorm Ove...,"For several years, a small nonprofit operating...","The founder of a Chicago youth group, who shar...",2019-09-06 22:44:19+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/09/06/arts/music/...,nyt://article/65b5b170-9325-53be-a95c-1edca8d2...,"The founder of a Chicago youth group, who shar..."
1,police_violence,"In an interview, Ms. Harris sought to paint he...",‘Trust Me’: Kamala Harris Makes Big Play on Cr...,Senator Kamala Harris of California released a...,"In an interview, Ms. Harris sought to paint he...",2019-09-09 09:30:07+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/09/09/us/politics...,nyt://article/009cafc5-8b75-5f39-a757-6e95a249...,"In an interview, Ms. Harris sought to paint he..."
2,police_violence,"Amber Guyger killed an unarmed black man, Both...",Former Dallas Police Officer Is Guilty of Murd...,DALLAS — The case was unusual from the very be...,"Amber Guyger killed an unarmed black man, Both...",2019-10-01 15:47:07+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/01/us/amber-gu...,nyt://article/2156cec9-000d-5c61-bf5f-bae2ce2b...,"Amber Guyger killed an unarmed black man, Both..."
3,police_violence,"In this week’s newsletter, Marc Stein has scoo...",The League They Told You Not to Worry About,Want more basketball in your inbox? Sign up fo...,"In this week’s newsletter, Marc Stein has scoo...",2019-10-02 16:39:23+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/02/sports/bask...,nyt://article/b1fca0ad-762b-52a9-8006-74599284...,"In this week’s newsletter, Marc Stein has scoo..."
4,police_violence,George Gascón resigned as San Francisco’s dist...,Two Prosecutors Were Shaped by 1980s Los Angel...,LOS ANGELES — Walking the beat as a cop in the...,George Gascón resigned as San Francisco’s dist...,2019-10-03 23:33:17+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/03/us/los-ange...,nyt://article/1b74b499-026e-5c3e-ac63-d9fe0ca9...,George Gascón resigned as San Francisco’s dist...
...,...,...,...,...,...,...,...,...,...,...,...
7987,noquery,Join us as we share stories of the Black exper...,From Many Comes One,,Join us as we share stories of the Black exper...,2021-07-22 18:26:42+00:00,None,BlackLivesMatter,https://www.nytimes.com/paidpost/starz/soul-of...,nyt://paidpost/ee158427-9a9b-5119-b874-72bd15d...,Join us as we share stories of the Black exper...
7988,destruction,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."
7989,race,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."
7990,noquery,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."


In [ ]:
data_df['type_of_material'].value_counts(normalize=False)

News                   5786
Op-Ed                   983
Interactive Feature     484
briefing                397
Review                  196
Letter                   46
Obituary (Obit)          31
Editorial                14
News Analysis            12
Video                    11
List                      7
Quote                     4
Correction                4
Name: type_of_material, dtype: int64

In [ ]:
data_df['type_of_material'].value_counts(normalize=False)

News                   5649
Op-Ed                   941
Interactive Feature     431
briefing                394
Review                  188
Letter                   46
Obituary (Obit)          31
Editorial                14
News Analysis            12
Video                    11
List                      7
Quote                     4
Correction                4
Name: type_of_material, dtype: int64

In [ ]:
data_df['filter'].value_counts(normalize=False)

noquery             2817
race                2060
police_violence     1541
destruction          891
violent_protest      459
peaceful_protest     224
Name: filter, dtype: int64

In [ ]:
data_df['person'].value_counts(normalize=False)

Black Lives Matter    7523
BLM                    276
BlackLivesMatter       193
Name: person, dtype: int64

In [ ]:
data_df.person.nunique()

3

In [ ]:
data_df._id.nunique()

2717

This returns the count of all unique values in the web_url column.


In [ ]:
data_df.web_url.nunique()

2717

In [ ]:
t = 0
for person, articles in ll:
    print(person, len(articles))
    t += len(articles)
print(t)

0


## BLM DATASET

In [ ]:
blm_df['type_of_material'].value_counts(normalize=False)

News                   6647
Op-Ed                  1009
briefing                493
Interactive Feature     439
Review                  234
Letter                   56
Obituary (Obit)          41
Editorial                15
Video                    14
News Analysis            11
Quote                     7
Correction                7
List                      7
Name: type_of_material, dtype: int64

In [ ]:
blm_df = data_df.query("person in ('Black Lives Matter','BLM', 'BlackLivesMatter')")
print(blm_df['person'].value_counts())


Black Lives Matter    7523
BLM                    276
BlackLivesMatter       193
Name: person, dtype: int64


In [ ]:
blm_df

,filter,abstract,headline,lead_paragraph,snippet,pub_date,type_of_material,person,web_url,_id,text
0,police_violence,"The founder of a Chicago youth group, who shar...",A Gift From the N.F.L. Ignites a Firestorm Ove...,"For several years, a small nonprofit operating...","The founder of a Chicago youth group, who shar...",2019-09-06 22:44:19+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/09/06/arts/music/...,nyt://article/65b5b170-9325-53be-a95c-1edca8d2...,"The founder of a Chicago youth group, who shar..."
1,police_violence,"In an interview, Ms. Harris sought to paint he...",‘Trust Me’: Kamala Harris Makes Big Play on Cr...,Senator Kamala Harris of California released a...,"In an interview, Ms. Harris sought to paint he...",2019-09-09 09:30:07+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/09/09/us/politics...,nyt://article/009cafc5-8b75-5f39-a757-6e95a249...,"In an interview, Ms. Harris sought to paint he..."
2,police_violence,"Amber Guyger killed an unarmed black man, Both...",Former Dallas Police Officer Is Guilty of Murd...,DALLAS — The case was unusual from the very be...,"Amber Guyger killed an unarmed black man, Both...",2019-10-01 15:47:07+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/01/us/amber-gu...,nyt://article/2156cec9-000d-5c61-bf5f-bae2ce2b...,"Amber Guyger killed an unarmed black man, Both..."
3,police_violence,"In this week’s newsletter, Marc Stein has scoo...",The League They Told You Not to Worry About,Want more basketball in your inbox? Sign up fo...,"In this week’s newsletter, Marc Stein has scoo...",2019-10-02 16:39:23+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/02/sports/bask...,nyt://article/b1fca0ad-762b-52a9-8006-74599284...,"In this week’s newsletter, Marc Stein has scoo..."
4,police_violence,George Gascón resigned as San Francisco’s dist...,Two Prosecutors Were Shaped by 1980s Los Angel...,LOS ANGELES — Walking the beat as a cop in the...,George Gascón resigned as San Francisco’s dist...,2019-10-03 23:33:17+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/03/us/los-ange...,nyt://article/1b74b499-026e-5c3e-ac63-d9fe0ca9...,George Gascón resigned as San Francisco’s dist...
...,...,...,...,...,...,...,...,...,...,...,...
7987,noquery,Join us as we share stories of the Black exper...,From Many Comes One,,Join us as we share stories of the Black exper...,2021-07-22 18:26:42+00:00,None,BlackLivesMatter,https://www.nytimes.com/paidpost/starz/soul-of...,nyt://paidpost/ee158427-9a9b-5119-b874-72bd15d...,Join us as we share stories of the Black exper...
7988,destruction,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."
7989,race,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."
7990,noquery,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."


In [ ]:
blm_df.to_csv(gdrive+"blm20221213.csv", encoding='utf-8')

In [ ]:
#blm_df = blm_df.drop(columns=['person'])
#blm_df = blm_df.drop(columns=['Unnamed: 0'])
blm_df = blm_df.drop_duplicates()

In [ ]:
blm_df

,filter,abstract,headline,lead_paragraph,snippet,pub_date,type_of_material,person,web_url,_id,text
0,police_violence,"The founder of a Chicago youth group, who shar...",A Gift From the N.F.L. Ignites a Firestorm Ove...,"For several years, a small nonprofit operating...","The founder of a Chicago youth group, who shar...",2019-09-06 22:44:19+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/09/06/arts/music/...,nyt://article/65b5b170-9325-53be-a95c-1edca8d2...,"The founder of a Chicago youth group, who shar..."
1,police_violence,"In an interview, Ms. Harris sought to paint he...",‘Trust Me’: Kamala Harris Makes Big Play on Cr...,Senator Kamala Harris of California released a...,"In an interview, Ms. Harris sought to paint he...",2019-09-09 09:30:07+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/09/09/us/politics...,nyt://article/009cafc5-8b75-5f39-a757-6e95a249...,"In an interview, Ms. Harris sought to paint he..."
2,police_violence,"Amber Guyger killed an unarmed black man, Both...",Former Dallas Police Officer Is Guilty of Murd...,DALLAS — The case was unusual from the very be...,"Amber Guyger killed an unarmed black man, Both...",2019-10-01 15:47:07+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/01/us/amber-gu...,nyt://article/2156cec9-000d-5c61-bf5f-bae2ce2b...,"Amber Guyger killed an unarmed black man, Both..."
3,police_violence,"In this week’s newsletter, Marc Stein has scoo...",The League They Told You Not to Worry About,Want more basketball in your inbox? Sign up fo...,"In this week’s newsletter, Marc Stein has scoo...",2019-10-02 16:39:23+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/02/sports/bask...,nyt://article/b1fca0ad-762b-52a9-8006-74599284...,"In this week’s newsletter, Marc Stein has scoo..."
4,police_violence,George Gascón resigned as San Francisco’s dist...,Two Prosecutors Were Shaped by 1980s Los Angel...,LOS ANGELES — Walking the beat as a cop in the...,George Gascón resigned as San Francisco’s dist...,2019-10-03 23:33:17+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/03/us/los-ange...,nyt://article/1b74b499-026e-5c3e-ac63-d9fe0ca9...,George Gascón resigned as San Francisco’s dist...
...,...,...,...,...,...,...,...,...,...,...,...
7987,noquery,Join us as we share stories of the Black exper...,From Many Comes One,,Join us as we share stories of the Black exper...,2021-07-22 18:26:42+00:00,None,BlackLivesMatter,https://www.nytimes.com/paidpost/starz/soul-of...,nyt://paidpost/ee158427-9a9b-5119-b874-72bd15d...,Join us as we share stories of the Black exper...
7988,destruction,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."
7989,race,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."
7990,noquery,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."


In [ ]:
blm_df._id.nunique()

2717

In [ ]:
blm_df

,filter,abstract,headline,lead_paragraph,snippet,pub_date,type_of_material,person,web_url,_id,text
0,police_violence,"The founder of a Chicago youth group, who shar...",A Gift From the N.F.L. Ignites a Firestorm Ove...,"For several years, a small nonprofit operating...","The founder of a Chicago youth group, who shar...",2019-09-06 22:44:19+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/09/06/arts/music/...,nyt://article/65b5b170-9325-53be-a95c-1edca8d2...,"The founder of a Chicago youth group, who shar..."
1,police_violence,"In an interview, Ms. Harris sought to paint he...",‘Trust Me’: Kamala Harris Makes Big Play on Cr...,Senator Kamala Harris of California released a...,"In an interview, Ms. Harris sought to paint he...",2019-09-09 09:30:07+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/09/09/us/politics...,nyt://article/009cafc5-8b75-5f39-a757-6e95a249...,"In an interview, Ms. Harris sought to paint he..."
2,police_violence,"Amber Guyger killed an unarmed black man, Both...",Former Dallas Police Officer Is Guilty of Murd...,DALLAS — The case was unusual from the very be...,"Amber Guyger killed an unarmed black man, Both...",2019-10-01 15:47:07+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/01/us/amber-gu...,nyt://article/2156cec9-000d-5c61-bf5f-bae2ce2b...,"Amber Guyger killed an unarmed black man, Both..."
3,police_violence,"In this week’s newsletter, Marc Stein has scoo...",The League They Told You Not to Worry About,Want more basketball in your inbox? Sign up fo...,"In this week’s newsletter, Marc Stein has scoo...",2019-10-02 16:39:23+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/02/sports/bask...,nyt://article/b1fca0ad-762b-52a9-8006-74599284...,"In this week’s newsletter, Marc Stein has scoo..."
4,police_violence,George Gascón resigned as San Francisco’s dist...,Two Prosecutors Were Shaped by 1980s Los Angel...,LOS ANGELES — Walking the beat as a cop in the...,George Gascón resigned as San Francisco’s dist...,2019-10-03 23:33:17+00:00,News,Black Lives Matter,https://www.nytimes.com/2019/10/03/us/los-ange...,nyt://article/1b74b499-026e-5c3e-ac63-d9fe0ca9...,George Gascón resigned as San Francisco’s dist...
...,...,...,...,...,...,...,...,...,...,...,...
7987,noquery,Join us as we share stories of the Black exper...,From Many Comes One,,Join us as we share stories of the Black exper...,2021-07-22 18:26:42+00:00,None,BlackLivesMatter,https://www.nytimes.com/paidpost/starz/soul-of...,nyt://paidpost/ee158427-9a9b-5119-b874-72bd15d...,Join us as we share stories of the Black exper...
7988,destruction,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."
7989,race,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."
7990,noquery,"The requirement, the first in the nation, was ...",All California Public High School Students Wil...,The hundreds of new laws that Gov. Gavin Newso...,"The requirement, the first in the nation, was ...",2021-10-14 12:49:29+00:00,briefing,BlackLivesMatter,https://www.nytimes.com/2021/10/14/us/californ...,nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...,"The requirement, the first in the nation, was ..."


In [ ]:
#blm_df.to_csv(gdrive+"blm_df.csv", encoding='utf-8')

In [ ]:
#blm_df = data_df.query("person in ('Black Lives Matter','BLM', 'BlackLivesMatter')")
print(blm_df['filter'].value_counts())

noquery             2624
race                1906
police_violence     1429
destruction          827
violent_protest      431
peaceful_protest     207
Name: filter, dtype: int64


In [ ]:
blm_df.to_csv(gdrive+"blm_20221212.csv", encoding='utf-8')

**2624** NYT materials include ('Black Lives Matter','BLM', 'BlackLivesMatter') keywords.

In [ ]:
blm_df._id

0       nyt://article/2fda17f5-5c95-54c2-83cd-b367dd5b...
1       nyt://article/1eb04e31-808a-5f52-beef-2cab693c...
2       nyt://article/e136d26e-f84d-5241-a4ef-8dbf07c1...
3       nyt://article/9d043e44-db40-576c-9af9-c3c0d158...
4       nyt://article/2cc0404d-70a5-5bbc-95af-6f59e0a8...
                              ...                        
2715    nyt://article/2aaf1a86-1a5e-516e-8c0b-e594dd87...
2717    nyt://article/38481888-d67a-5b2f-9bff-bc18d706...
2719    nyt://article/0f0f2245-c429-5665-92e2-197a285a...
2721    nyt://article/ba4d8f07-3c7f-55aa-ad4d-c0c6b2c2...
2722    nyt://article/b91cbe3f-3bf5-5ee6-945e-f39da6fd...
Name: _id, Length: 2624, dtype: object

In [ ]:
blm_list = set(blm_df._id.to_list())

In [ ]:
pivot_list = []
for person in Person_list:
    d = {}
    person_df = data_df.query("person in ('%s')" % person)
    person_ids = set(person_df._id.to_list())
    person_total = len(person_ids)
    person_in_black_lives_matter = len(blm_list.intersection(person_ids))
    d["person"] = person
    d["total_numberof_news"] = person_total
    d["in_blm_news"] = person_in_black_lives_matter
    d["not_in_blm_news"] =  person_total - person_in_black_lives_matter
    pivot_list.append(d)
    print(person, person_total, person_in_black_lives_matter, person_total - person_in_black_lives_matter)
pivot_df = pd.DataFrame(pivot_list)
pivot_df



Black Lives Matter 2570 2570 0
BLM 95 95 0
BlackLivesMatter 58 58 0
michael wayne jackson 0 0 0
arnell stewart 0 0 0
fanta bility 2 0 2
alvin motley 0 0 0
ryan leroux 0 0 0
winston smith 4 2 2
latoya denise james 0 0 0
andrew brown 32 4 28
makhia bryant 0 0 0
matthew williams 15 1 14
daunte wright 145 16 129
james lionel johnson 0 0 0
dominique williams 0 0 0
donovon lynch 0 0 0
marvin scott 3 0 3
jenoah donald 1 1 0
patrick warren 6 0 6
xzavier hill 0 0 0
robert howard 5 0 5
vincent belmonte 0 0 0
monica goods 1 0 1
bennie edwards 2 0 2
casey goodson 10 2 8
aiden ellison 2 0 2
quawan charles 2 0 2
kevin peterson 1 0 1
walter wallace 43 25 18
jonathan price 1 0 1
kurt reinhold 0 0 0
dijon kizzee 6 2 4
damian daniels 0 0 0
anthony mcclain 2 1 1
julian lewis 3 1 2
maurice abisid 0 0 0
brayla stone 0 0 0
rayshard brooks 125 39 86
priscilla slater 0 0 0
robert forbes 0 0 0
kamal flowers 0 0 0
jamel floyd 3 2 1
david mcatee 23 9 14
james scurlock 5 2 3
calvin horton 0 0 0
tony mcdade 41 35 

,person,total_numberof_news,in_blm_news,not_in_blm_news
0,Black Lives Matter,2570,2570,0
1,BLM,95,95,0
2,BlackLivesMatter,58,58,0
3,michael wayne jackson,0,0,0
4,arnell stewart,0,0,0
...,...,...,...,...
58,lionel morris,0,0,0
59,jaquyn light,0,0,0
60,william green,3,0,3
61,darius tarver,2,1,1


In [ ]:
pivot_df.to_csv(gdrive+"blm_list.csv", encoding='utf-8')

# BLM SUPPORTERS' DEMANDS

In [ ]:
#Person_list = ['Eric Garner', 'Michael Brown', 'Ahmaud Arbery', 'Rittenhouse']
Person_list = ['Black Lives Matter', 'BLM', 'BlackLivesMatter']
#type_material = ['News', 'Editorial', 'Op-Ed']
ll = []

list_2021=['michael wayne jackson','arnell stewart','fanta bility','alvin motley','ryan leroux','winston smith','latoya denise james','andrew brown','makhia bryant','matthew williams','daunte wright','james lionel johnson','dominique williams','donovon lynch','marvin scott','jenoah donald','patrick warren','xzavier hill','robert howard','vincent belmonte']
list_2020=['monica goods','bennie edwards','casey goodson','aiden ellison','quawan charles','kevin peterson','walter wallace','jonathan price','kurt reinhold','dijon kizzee','damian daniels','anthony mcclain','julian lewis','maurice abisid','brayla stone','rayshard brooks','priscilla slater','robert forbes','kamal flowers','jamel floyd','david mcatee','james scurlock','calvin horton','tony mcdade','dion johnson','george floyd','maurice gordon','cornelius fredericks','steven taylor','daniel prude','breonna taylor','barry gedeus','manuel ellis','reginald payne','ahmaud arbery','lionel morris','jaquyn light','william green','darius tarver','miciah lee']
#Person_list += list_2021
#Person_list += list_2020
#Person_list = list_2020
MAX_DATE = datetime.date(2021, 12, 31)
START_DATE = datetime.date(2020, 1, 1)
victims = list_2021 + list_2020
filter_kewyords = {
    "defund the police": ['defund the police'],
    "abolish the police": ['abolish the police'],
    "police reform": ['police reform'],
    "racisim": ['racism'],
    "police reform": ['police reform'],
    "systemic racism": ['systemic racism'],
    "racial inequality": ['racial inequality'],
    "racial injustice": ["racial injustice"],
    "white supremacy": ["white supremacy"],
    "police brutality": ["police brutality"],
    "i can’t breathe": ["i can’t breathe"],
    "racial equality": ['racial equality'],
    "racial justice": ["racial justice"],
    "justice for": list(map(lambda victim: "justice for " + victim, victims))
}

def generate_fq(filter_list):
   if len(filter_list) == 1:
       return 'body :("%s")' % filter_list[0]
   start = "body :("
   end = ")"
   filter_kw = '("%s")'
   filter_kw_list = []
   for y in filter_list:
        fq_key = filter_kw % y
        filter_kw_list.append(fq_key)
   middle = " OR ".join(filter_kw_list)
   return start + middle + end

filters = {}
for kw in filter_kewyords:
    filters[kw] = generate_fq(filter_kewyords[kw])

filters


In [ ]:
import itertools

nyt = NYTAPI(my_key, parse_dates=True)

#Person_list = ['Eric Garner', 'Michael Brown', 'Ahmaud Arbery', 'Rittenhouse']
Person_list = ['Black Lives Matter', 'BLM', 'BlackLivesMatter']
#type_material = ['News', 'Editorial', 'Op-Ed']
ll = []

#list_2021=['michael wayne jackson','arnell stewart','fanta bility','alvin motley','ryan leroux','winston smith','latoya denise james','andrew brown','makhia bryant','matthew williams','daunte wright','james lionel johnson','dominique williams','donovon lynch','marvin scott','jenoah donald','patrick warren','xzavier hill','robert howard','vincent belmonte']
#list_2020=['monica goods','bennie edwards','casey goodson','aiden ellison','quawan charles','kevin peterson','walter wallace','jonathan price','kurt reinhold','dijon kizzee','damian daniels','anthony mcclain','julian lewis','maurice abisid','brayla stone','rayshard brooks','priscilla slater','robert forbes','kamal flowers','jamel floyd','david mcatee','james scurlock','calvin horton','tony mcdade','dion johnson','george floyd','maurice gordon','cornelius fredericks','steven taylor','daniel prude','breonna taylor','barry gedeus','manuel ellis','reginald payne','ahmaud arbery','lionel morris','jaquyn light','william green','darius tarver','miciah lee']
#Person_list += list_2021
#Person_list += list_2020
#Person_list = list_2020
MAX_DATE = datetime.date(2021, 12, 31)
START_DATE = datetime.date(2020, 1, 1)

for i in Person_list:
    timediff = datetime.timedelta(days=60)
    start_date = START_DATE
    end_date = min(MAX_DATE, start_date + timediff)
    all_articles = []
    while start_date < MAX_DATE:
        for key, fq in filters.items():
            if fq:
                options = {"sort": "oldest",
                            "fq": fq}
            else:
                options = {"sort": "oldest",}
            articles = nyt.article_search(
                query = '"'+ i + '"',
                results = 2000,
                dates = {
                  "begin": start_date,
                  "end": end_date
                },
                options = options
            )
            print(i, len(articles), start_date, end_date, key)
            all_articles.append((key, articles))
        start_date = end_date + datetime.timedelta(days=1)
        end_date = min(MAX_DATE, start_date + timediff)
    ll.append((i, all_articles))

In [ ]:
#column in_text will show “True” for a match in head, abstract, lead_paragraph or snippet, “False” if it is not a match.

def articles_to_df_for_text(all_articles):
    l = []
    for person, tag_level in all_articles:
        for tag, articles in tag_level:
            for article in articles:
                text_list = []
                new_article = {
                    'filter': tag,
                    'abstract': article["abstract"],
                    "headline": article["headline"]["main"],
                    "lead_paragraph": article["lead_paragraph"],
                    "snippet": article["snippet"],
                    "pub_date": article["pub_date"],
                    "type_of_material": article.get("type_of_material"),
                    "person": person,
                    "web_url": article["web_url"],
                    '_id': article['_id']
                }
                for k in ["abstract", "headline", "lead_paragraph", "snippet"]:
                    v = new_article[k]
                    if v:
                        v = v.strip()
                    new_article[k] = v
                    if v not in text_list:
                        text_list.append(v)
                new_article["text"] = ' '.join(text_list)
                if any(
                      map(lambda tag_filter: tag_filter.lower() in new_article["text"].lower(),
                          filter_kewyords[tag])):
                    new_article["in_text"] = True
                else:
                    new_article["in_text"] = False
                l.append(new_article)
    data_df = pd.DataFrame(l)
    return data_df

data_df = articles_to_df_for_text(ll)
with open(gdrive+"nyt_articles0913_tag.p", "wb") as f:
    f.write(pickle.dumps(data_df))
data_df

In [ ]:
data_df.to_csv(gdrive+"nyt_demands13.csv", encoding='utf-8')

In [ ]:
print(data_df['filter'].value_counts())

In [ ]:
data_df._id.nunique()

In [ ]:
#demand_df = data_df.drop(columns=['person'])
demand_df = demand_df.drop_duplicates()


In [ ]:
demand_df

In [ ]:
print(demand_df['in_text'].value_counts())

In [ ]:
in_text_df = demand_df.query("in_text")


In [ ]:
in_text_df._id.nunique()

In [ ]:
print(in_text_df['filter'].value_counts())

In [ ]:
text_df = demand_df.query("~in_text")


In [ ]:
text_df

In [ ]:
print(text_df['filter'].value_counts())